In [23]:
!pip install -q pyomo

In [24]:
from pyomo.environ import *
import numpy as np

***Question 1***

Write a mathematical model so that you can maximize your credits to be accumulated
this semester. Explain how to design the model variables and indicate which variables are
integers.


***Solution***

There is two options for each and every course either we take it or drop So we represent taking the course as 1 and not taking the course as 0

So all varaibles $C_0$,$C_1$,$C_2$,$C_3$,$C_4$,$C_5$,$C_6$,$C_7$,$C_8$,$C_9$ are binary that means they take only one vaule among {0,1}


In [25]:
m1 = ConcreteModel()

In [26]:
N = 10 # No of variables
col_indices = np.arange(N)

In [27]:
m1.c = Var(col_indices,domain = Binary)#Defining all variables as binary

In [28]:
m1.pprint()

1 Set Declarations
    c_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}

1 Var Declarations
    c : Size=10, Index=c_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  None :     1 : False :  True : Binary
          9 :     0 :  None :     1 : False :  True : Binary

2 Declarations: c_index c


***Question1 & 2Solution***

*   objective function description was :: "you can maximize your credits to be accumulated this semester." :: So maximize 4$c_0$+3$c_1$+6$c_2$+5$c_3$+2$c_4$+6$c_5$+4$c_6$+8$c_7$+8$c_8$+6$c_9$

***Formulating constraints:***

* You must gather at least 26 credits this semester::
  
  so 4$c_0$+3$c_1$+6$c_2$+5$c_3$+2$c_4$+6$c_5$+4$c_6$+8$c_7$+8$c_8$+6$c_9$>=26

  so we convert it into <= by multiplying '-ve' sign on both sides so we can easily define in the code (by taking all <=)

  so Constraint 1 is :: -4$c_0$-3$c_1$-6$c_2$-5$c_3$-2$c_4$-6$c_5$-4$c_6$-8$c_7$-8$c_8$-6$c_9$<=-26

* You must register for at least 4 courses and can register for up to a maximum of 8 courses this semester ::

so 4 < = $c_0$+$c_1$+$c_2$+$c_3$+$c_4$+$c_5$+$c_6$+$c_7$+$c_8$+$c_9$ <= 8

So constraint 2 is ::

$c_0$+$c_1$+$c_2$+$c_3$+$c_4$+$c_5$+$c_6$+$c_7$+$c_8$+$c_9$ <= 8

constraint 3 is ::

-$c_0$-$c_1$-$c_2$-$c_3$-$c_4$-$c_5$-$c_6$-$c_7$-$c_8$-$c_9$ <= -4

* Courses C2 and C7 run during the same slots : so we can take only one course among $c_2$,$c_7$
  
  Constraint 4 is :: $c_2$+$c_7$<=1

* Courses C1 and C9 run during the same slots : so we can take only one course among $c_1$,$c_9$

Constraint 5 is :: $c_2$+$c_7$<=1

* If you register for course C2 you cannot register for course C8 

Constraint 6 is :: $c_2$+$c_8$ <= 1

* If you register for courses C9 you must register for course C4 : so if we take

c9 then to satisfy the equation C4 will be equal to 1

Constraint 7 is :: $c_9$-$c_4$ < = 0

* Among the courses C0, C1, C2, C3, you must register for at least one course and can only register for at most two courses.

So  Constraint 8 is :: $c_0$+$c_1$+$c_2$+$c_3$ <= 2

  Constraint 9 is ::  -$c_0$-$c_1$-$c_2$-$c_3$ <= -1

* Among the courses C5, C9, you can only register at most one course.

So Constraint 10 is :: $c_5$+$c_9$ < = 1

* Among the courses C6, C8, C9, you can only register for at most two courses

So constraint 11 is :: $c_6$+$c_8$+$c_9$ < = 2




In [29]:
cost_coeff = np.array([4,3,6,5,2,6,4,8,8,6])
m1.cost = Objective(expr =summation(cost_coeff,m1.c),sense = maximize) 
constr_coeff = np.array([[-4,-3,-6,-5,-2,-6,-4,-8,-8,-6],
                        [1]*10,
                        [-1]*10,
                        [0,0,1,0,0,0,0,1,0,0],
                        [0,1,0,0,0,0,0,0,0,1],
                        [0,0,1,0,0,0,0,0,1,0],
                        [0,0,0,0,-1,0,0,0,0,1],
                        [1,1,1,1,0,0,0,0,0,0],
                        [-1,-1,-1,-1,0,0,0,0,0,0],
                        [0,0,0,0,0,1,0,0,0,1],
                        [0,0,0,0,0,0,1,0,1,1]])
M = 11
row_indices = np.arange(M)

In [30]:
m1.constraints = ConstraintList()

In [31]:
constr_rhs = [-26,8,-4,1,1,1,0,2,-1,1,2]
for i in row_indices:
  m1.constraints.add(expr = summation(constr_coeff[i],m1.c)<=constr_rhs[i])

In [32]:
m1.pprint()

2 Set Declarations
    c_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

1 Var Declarations
    c : Size=10, Index=c_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  None :   

In [33]:
!apt-get install -y -qq coinor-cbc

In [34]:
opt_cbc = SolverFactory('cbc')
result = opt_cbc.solve(m1)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 37.0
  Upper bound: 37.0
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 10
  Number of binary variables: 10
  Number of integer variables: 10
  Number of nonzeros: 10
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.02824091911315918
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [35]:
# display solution
print('\nObjective = ', m1.cost())

print('\nDecision Variables')
for i in col_indices:
  print('c[',i,']= ', m1.c[i].value)
m1.constraints.display()


Objective =  37.0

Decision Variables
c[ 0 ]=  1.0
c[ 1 ]=  0.0
c[ 2 ]=  0.0
c[ 3 ]=  1.0
c[ 4 ]=  1.0
c[ 5 ]=  1.0
c[ 6 ]=  1.0
c[ 7 ]=  1.0
c[ 8 ]=  1.0
c[ 9 ]=  0.0
constraints : Size=11
    Key : Lower : Body  : Upper
      1 :  None : -37.0 : -26.0
      2 :  None :   7.0 :   8.0
      3 :  None :  -7.0 :  -4.0
      4 :  None :   1.0 :   1.0
      5 :  None :   0.0 :   1.0
      6 :  None :   1.0 :   1.0
      7 :  None :  -1.0 :   0.0
      8 :  None :   2.0 :   2.0
      9 :  None :  -2.0 :  -1.0
     10 :  None :   1.0 :   1.0
     11 :  None :   2.0 :   2.0


*** Question 3 Solution ***

We got optimum solution 


Objective value  =  37.0

Decision Variables 
* c[ 0 ] =  1
* c[ 1 ] =  0
* c[ 2 ] =  0
* c[ 3 ] =  1
* c[ 4 ] =  1
* c[ 5 ] =  1
* c[ 6 ] =  1
* c[ 7 ] =  1
* c[ 8 ] =  1
* c[ 9 ] =  0

***Question 4*** 

remove the restrictions that the variables in the above problem are integers

In [36]:
m2 = ConcreteModel()
N = 10 # No of variables
col_indices = np.arange(N)
m2.c = Var(col_indices)#removing binary restriction
cost_coeff = np.array([4,3,6,5,2,6,4,8,8,6])
m2.cost = Objective(expr =summation(cost_coeff,m2.c),sense = maximize) 
for i in col_indices:#setting lower and upper bounds as domains are reals
  m2.c[i].setub(1)
  m2.c[i].setlb(0)
constr_coeff = np.array([[-4,-3,-6,-5,-2,-6,-4,-8,-8,-6],
                        [1]*10,
                        [-1]*10,
                        [0,0,1,0,0,0,0,1,0,0],
                        [0,1,0,0,0,0,0,0,0,1],
                        [0,0,1,0,0,0,0,0,1,0],
                        [0,0,0,0,-1,0,0,0,0,1],
                        [1,1,1,1,0,0,0,0,0,0],
                        [-1,-1,-1,-1,0,0,0,0,0,0],
                        [0,0,0,0,0,1,0,0,0,1],
                        [0,0,0,0,0,0,1,0,1,1]])
M = 11
row_indices = np.arange(M)
m2.constraints = ConstraintList()
constr_rhs = [-26,8,-4,1,1,1,0,2,-1,1,2]
for i in row_indices:
  m2.constraints.add(expr = summation(constr_coeff[i],m2.c)<=constr_rhs[i])
m2.pprint()

2 Set Declarations
    c_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

1 Var Declarations
    c : Size=10, Index=c_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True :  Reals
          1 :     0 :  None :     1 : False :  True :  Reals
          2 :     0 :  None :     1 : False :  True :  Reals
          3 :     0 :  None :     1 : False :  True :  Reals
          4 :     0 :  None :     1 : False :  True :  Reals
          5 :     0 :  None :     1 : False :  True :  Reals
          6 :     0 :  None :     1 : False :  True :  Reals
          7 :     0 :  None :     1 : False :  True :  Reals
          8 :     0 :  None :   

In [37]:
result = opt_cbc.solve(m2)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 37.0
  Upper bound: 37.0
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 11
  Number of nonzeros: 10
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 3
  Error rc: 0
  Time: 0.029267549514770508
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [38]:
# display solution
print('\nObjective = ', m2.cost())

print('\nDecision Variables')
for i in col_indices:
  print('c[',i,'] = ', m2.c[i].value)
m2.constraints.display()


Objective =  37.0

Decision Variables
c[ 0 ] =  1.0
c[ 1 ] =  0.0
c[ 2 ] =  0.0
c[ 3 ] =  1.0
c[ 4 ] =  1.0
c[ 5 ] =  1.0
c[ 6 ] =  1.0
c[ 7 ] =  1.0
c[ 8 ] =  1.0
c[ 9 ] =  0.0
constraints : Size=11
    Key : Lower : Body  : Upper
      1 :  None : -37.0 : -26.0
      2 :  None :   7.0 :   8.0
      3 :  None :  -7.0 :  -4.0
      4 :  None :   1.0 :   1.0
      5 :  None :   0.0 :   1.0
      6 :  None :   1.0 :   1.0
      7 :  None :  -1.0 :   0.0
      8 :  None :   2.0 :   2.0
      9 :  None :  -2.0 :  -1.0
     10 :  None :   1.0 :   1.0
     11 :  None :   2.0 :   2.0


***Question 4 Solution***

After removing the restriction of binary and given bounds to varibles We got this optimum solution 


Objective value  =  37.0

Decision Variables 
* c[ 0 ] =  1
* c[ 1 ] =  0
* c[ 2 ] =  0
* c[ 3 ] =  1
* c[ 4 ] =  1
* c[ 5 ] =  1
* c[ 6 ] =  1
* c[ 7 ] =  1
* c[ 8 ] =  1
* c[ 9 ] =  0

***Question 5 Solution*** 

Since the lpp is maximization problem , coefficients are also positive and  the solver will take maximum value if possible and maximum value using binary is 1 and maximum value using reals with upper bounds as 1 the maximum value is same here.
So we are getting the same  answer as maximum of binary and bounds because they are same

***Question 6***
Suppose a new course C10 with 12 credits is introduced, but the course timing for C10 overlaps with the course timings of C5, C6, how will you change your model to include this new constraint

So Constraint 12 : $c_5$+$c_6$+$c_{10}$ <=1

In [39]:
m3 = ConcreteModel()
N = 11 # No of variables
col_indices = np.arange(N)
m3.c = Var(col_indices,domain = Binary)
cost_coeff = np.array([4,3,6,5,2,6,4,8,8,6,12])
m3.cost = Objective(expr =summation(cost_coeff,m3.c),sense = maximize) 
constr_coeff = np.array([[-4,-3,-6,-5,-2,-6,-4,-8,-8,-6,-12],
                        [1]*11,
                        [-1]*11,
                        [0,0,1,0,0,0,0,1,0,0,0],
                        [0,1,0,0,0,0,0,0,0,1,0],
                        [0,0,1,0,0,0,0,0,1,0,0],
                        [0,0,0,0,-1,0,0,0,0,1,0],
                        [1,1,1,1,0,0,0,0,0,0,0],
                        [-1,-1,-1,-1,0,0,0,0,0,0,0],
                        [0,0,0,0,0,1,0,0,0,1,0],
                        [0,0,0,0,0,0,1,0,1,1,0],
                        [0,0,0,0,0,1,1,0,0,0,1]])
M = 12
row_indices = np.arange(M)
m3.constraints = ConstraintList()
constr_rhs = [-26,8,-4,1,1,1,0,2,-1,1,2,1]
for i in row_indices:
  m3.constraints.add(expr = summation(constr_coeff[i],m3.c)<=constr_rhs[i])
m3.pprint()

2 Set Declarations
    c_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

1 Var Declarations
    c : Size=11, Index=c_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  N

In [40]:
result = opt_cbc.solve(m3)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 45.0
  Upper bound: 45.0
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 11
  Number of binary variables: 11
  Number of integer variables: 11
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.01
  Wallclock time: 0.01
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.0460209846496582
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [41]:
# display solution
print('\nObjective = ', m3.cost())

print('\nDecision Variables')
for i in col_indices:
  print('c[',i,'] = ', m3.c[i].value)
m3.constraints.display()


Objective =  45.0

Decision Variables
c[ 0 ] =  1.0
c[ 1 ] =  0.0
c[ 2 ] =  0.0
c[ 3 ] =  1.0
c[ 4 ] =  1.0
c[ 5 ] =  0.0
c[ 6 ] =  0.0
c[ 7 ] =  1.0
c[ 8 ] =  1.0
c[ 9 ] =  1.0
c[ 10 ] =  1.0
constraints : Size=12
    Key : Lower : Body  : Upper
      1 :  None : -45.0 : -26.0
      2 :  None :   7.0 :   8.0
      3 :  None :  -7.0 :  -4.0
      4 :  None :   1.0 :   1.0
      5 :  None :   1.0 :   1.0
      6 :  None :   1.0 :   1.0
      7 :  None :   0.0 :   0.0
      8 :  None :   2.0 :   2.0
      9 :  None :  -2.0 :  -1.0
     10 :  None :   1.0 :   1.0
     11 :  None :   2.0 :   2.0
     12 :  None :   1.0 :   1.0


***Question 7 Solution***

We got optimal solution and we got


Objective =  45.0

Decision Variables:
* c[ 0 ]  =  1
* c[ 1 ]  =  0
* c[ 2 ]  =  0
* c[ 3 ]  =  1
* c[ 4 ]  =  1
* c[ 5 ]  =  0
* c[ 6 ]  =  0
* c[ 7 ]  =  1
* c[ 8 ]  =  1
* c[ 9 ]  =  1
* c[ 10 ] =  1



***Question8:***


In [42]:
m4 = ConcreteModel()
N = 11 # No of variables
col_indices = np.arange(N)
m4.c = Var(col_indices)#removing binary restriction
for i in col_indices:#setting lower and upper bounds as domains are reals
  m4.c[i].setub(1)
  m4.c[i].setlb(0)
cost_coeff = np.array([4,3,6,5,2,6,4,8,8,6,12])
m4.cost = Objective(expr =summation(cost_coeff,m4.c),sense = maximize) 
constr_coeff = np.array([[-4,-3,-6,-5,-2,-6,-4,-8,-8,-6,-12],
                        [1]*11,
                        [-1]*11,
                        [0,0,1,0,0,0,0,1,0,0,0],
                        [0,1,0,0,0,0,0,0,0,1,0],
                        [0,0,1,0,0,0,0,0,1,0,0],
                        [0,0,0,0,-1,0,0,0,0,1,0],
                        [1,1,1,1,0,0,0,0,0,0,0],
                        [-1,-1,-1,-1,0,0,0,0,0,0,0],
                        [0,0,0,0,0,1,0,0,0,1,0],
                        [0,0,0,0,0,0,1,0,1,1,0],
                        [0,0,0,0,0,1,1,0,0,0,1]])
M = 12
row_indices = np.arange(M)
m4.constraints = ConstraintList()
constr_rhs = [-26,8,-4,1,1,1,0,2,-1,1,2,1]
for i in row_indices:
  m4.constraints.add(expr = summation(constr_coeff[i],m4.c)<=constr_rhs[i])
m4.pprint()

2 Set Declarations
    c_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   12 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

1 Var Declarations
    c : Size=11, Index=c_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True :  Reals
          1 :     0 :  None :     1 : False :  True :  Reals
          2 :     0 :  None :     1 : False :  True :  Reals
          3 :     0 :  None :     1 : False :  True :  Reals
          4 :     0 :  None :     1 : False :  True :  Reals
          5 :     0 :  None :     1 : False :  True :  Reals
          6 :     0 :  None :     1 : False :  True :  Reals
          7 :     0 :  None :     1 : False :  True :  Reals
          8 :     0 :  N

In [43]:
result = opt_cbc.solve(m4)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 45.0
  Upper bound: 45.0
  Number of objectives: 1
  Number of constraints: 13
  Number of variables: 12
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 6
  Error rc: 0
  Time: 0.028636455535888672
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [44]:
# display solution
print('\nObjective = ', m4.cost())

print('\nDecision Variables')
for i in col_indices:
  print('c[',i,'] = ', m4.c[i].value)
m4.constraints.display()


Objective =  45.0

Decision Variables
c[ 0 ] =  1.0
c[ 1 ] =  0.0
c[ 2 ] =  0.0
c[ 3 ] =  1.0
c[ 4 ] =  1.0
c[ 5 ] =  0.0
c[ 6 ] =  0.0
c[ 7 ] =  1.0
c[ 8 ] =  1.0
c[ 9 ] =  1.0
c[ 10 ] =  1.0
constraints : Size=12
    Key : Lower : Body  : Upper
      1 :  None : -45.0 : -26.0
      2 :  None :   7.0 :   8.0
      3 :  None :  -7.0 :  -4.0
      4 :  None :   1.0 :   1.0
      5 :  None :   1.0 :   1.0
      6 :  None :   1.0 :   1.0
      7 :  None :   0.0 :   0.0
      8 :  None :   2.0 :   2.0
      9 :  None :  -2.0 :  -1.0
     10 :  None :   1.0 :   1.0
     11 :  None :   2.0 :   2.0
     12 :  None :   1.0 :   1.0


***Question 8 Solution***

After making the varibles free of binary and adding bounds this is the solution


Objective =  45.0

Decision Variables:
* c[ 0 ]  =  1
* c[ 1 ]  =  0
* c[ 2 ]  =  0
* c[ 3 ]  =  1
* c[ 4 ]  =  1
* c[ 5 ]  =  0
* c[ 6 ]  =  0
* c[ 7 ]  =  1
* c[ 8 ]  =  1
* c[ 9 ]  =  1
* c[ 10 ] =  1

"Since the lpp is maximization problem , coefficients are also positive and  the solver will take maximum value if possible and maximum value using binary is 1 and maximum value using reals with upper bounds as 1 the maximum value is same here.
So we are getting the same  answer as maximum of binary and bounds because they are same"